In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import datetime as dt
from pathlib import Path

# Base data directory
DATA_DIR = Path("data")

c:\Users\iauge\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
acs_dtypes = {
    "geo_id": str,
    "year": int,
    "median_income": float,
    "total_pop": int,
    "median_age": float,
    "white_pop": float,
    "black_pop": float,
    "hispanic_pop": float,
    "asian_pop": float,
    "bachelors_degree_or_higher_25_64": float,
    "less_than_high_school_graduate": float,
    "some_college_and_associates_degree": float,
    "percent_income_spent_on_rent": float,
    "different_house_year_ago_same_city": float,
    "different_house_year_ago_different_city": float,
    "median_rent": float,
    "rent_over_50_percent": float,    
    "poverty": float,
    "gini_index": float
}

acs_df = pd.read_csv(DATA_DIR / "acs_multi_year_panel.csv", dtype=acs_dtypes)

# Sanity check FIPS formatting
acs_df["geo_id"] = acs_df["geo_id"].str.zfill(5)

# Drop nulls in income (2 rows)
acs_df = acs_df.dropna(subset=["median_income"])

# Fill minor gaps with  less than 10 nulls
minor_cols = ["median_rent", "percent_income_spent_on_rent", "rent_over_50_percent", "poverty", "gini_index"]
acs_df[minor_cols] = acs_df[minor_cols].fillna(acs_df[minor_cols].median())

acs_df.head()

In [ ]:
weather_dtypes = {
    "station_id": str,
    "year": int,
    "quarter": int,
    "avg_temp": float,
    "max_temp": float,
    "total_precip": float,
    "heat_days_90F": int,
    "hail_days": int,
    "thunder_days": int,
    "tornado_days": int,
    "num_days": int
}

# Load GSOD aggregates
weather_df = pd.read_csv(DATA_DIR / "gsod_quarterly_aggregates.csv", dtype=weather_dtypes)

weather_df.head()

In [ ]:
station_dtype = {
    "usaf": str,
    "wban": str,
    "name": str,
    "country": str,
    "state": str,
    "lat": float,
    "lon": float,
    "elev": str  # keep as str to preserve sign and format
    }

# Load file
stations_df = pd.read_csv(
    DATA_DIR / "noaa_stations.csv",
    dtype=station_dtype,
    parse_dates=["begin", "end"]
)

stations_df.head()

In [5]:
storm_dtypes = {
    "geo_id": str,
    "year": int,
    "quarter": int,
    "event_type": str,
    "num_events": int,
    "total_property_damage": float,
    "total_crop_damage": float,
    "total_injuries": int,
    "total_deaths": int
}

storm_df = pd.read_csv(DATA_DIR / "storm_events_by_county_quarter.csv", dtype=storm_dtypes)

print("✅ Loaded storm event data:", storm_df.shape)
storm_df.head()

✅ Loaded storm event data: (330416, 9)


,geo_id,year,quarter,event_type,num_events,total_property_damage,total_crop_damage,total_injuries,total_deaths
0,33007,2000,2,debris flow,1,0.0,0.0,0,0
1,36001,2000,2,debris flow,1,500000.0,0.0,0,0
2,51009,2000,1,dust devil,1,3000.0,0.0,0,0
3,32031,2000,3,dust devil,1,1000.0,0.0,0,0
4,21015,2000,1,flash flood,3,25000.0,0.0,0,0


In [6]:
counties = gpd.read_file(r"cb_2023_us_county_5m\cb_2023_us_county_5m.shp")

# Project to WGS84 (matches lat/lon)
counties = counties.to_crs("EPSG:4326")

# Create a unique FIPS code for join
counties["geo_id"] = counties["STATEFP"] + counties["COUNTYFP"]

# Select relevant columns
counties = counties[["geo_id", "NAME", "STATEFP", "COUNTYFP", "geometry"]]
counties.head()

,geo_id,NAME,STATEFP,COUNTYFP,geometry
0,13065,Clinch,13,065,"POLYGON ((-82.9705 30.94981, -82.97125 31.1839..."
1,21083,Graves,21,083,"POLYGON ((-88.81724 36.76775, -88.81323 36.773..."
2,37177,Tyrrell,37,177,"POLYGON ((-76.4056 35.78629, -76.35367 35.8613..."
3,38013,Burke,38,013,"POLYGON ((-102.93896 48.99928, -102.67765 48.9..."
4,40059,Harper,40,059,"POLYGON ((-100.00371 36.79697, -100.00256 37.0..."


In [7]:
# Convert stations_df to a GeoDataFrame
stations_gdf = gpd.GeoDataFrame(
    stations_df,
    geometry=gpd.points_from_xy(stations_df["lon"], stations_df["lat"]),
    crs="EPSG:4326"
)

# Perform spatial join (retains station geometry by default)
joined = gpd.sjoin(
    stations_gdf,
    counties[["geo_id", "NAME", "geometry"]],
    how="left",
    predicate="within"
)

# Rename county geometry to keep it
joined = joined.rename(columns={"geometry": "station_geometry", "NAME": "county_name"})

# Add back county geometry explicitly
joined = joined.merge(
    counties[["geo_id", "geometry"]].rename(columns={"geometry": "county_geometry"}),
    on="geo_id",
    how="left"
)

# Keep what's relevant, now with county geometry
stations_with_fips = gpd.GeoDataFrame(
    joined[["station_id", "geo_id", "county_name", "lat", "lon", "county_geometry"]],
    geometry="county_geometry",
    crs="EPSG:4326"
)

# Filter out non-US stations
weather_df = weather_df[weather_df["station_id"].isin(stations_with_fips["station_id"])]

# Merge Weather and Station data to eventually merge with ACS data
weather_df = weather_df.merge(stations_with_fips[["station_id", "geo_id", "county_name", "county_geometry"]], on="station_id", how="left")

In [8]:
def pivot_storm_events_by_type(
    storm_df: pd.DataFrame,
    top_n: int = None
) -> pd.DataFrame:
    
    storm_df = storm_df.copy()
    metrics = [
        "num_events",
        "total_injuries",
        "total_deaths",
        "total_property_damage",
        "total_crop_damage"
    ]

    storm_df[metrics] = storm_df[metrics].fillna(0)

    # Auto-select most impactful event types
    if top_n is not None:
        summary = storm_df.groupby("event_type").agg(
            total_deaths=("total_deaths", "sum"),
            total_damage=("total_property_damage", "sum")
        )
        summary["combined_impact"] = summary["total_deaths"] + (summary["total_damage"] / 1_000_000)  # scale damage
        top_event_types = summary.sort_values(by="combined_impact", ascending=False).head(top_n).index.tolist()
        storm_df = storm_df[storm_df["event_type"].isin(top_event_types)]

    # Pivot the filtered data
    pivot = storm_df.pivot_table(
        index=["geo_id", "year", "quarter"],
        columns="event_type",
        values=metrics,
        aggfunc="sum",
        fill_value=0
    )

    pivot.columns = [
        f"{metric}_{event_type.replace(' ', '_').lower()}"
        for metric, event_type in pivot.columns
    ]

    return pivot.reset_index()


In [9]:
# Run severe weather data through function to flatten data and highlight most impactful events
storm_wide = pivot_storm_events_by_type(storm_df, top_n=10)
print(storm_wide.columns)

Index(['geo_id', 'year', 'quarter', 'num_events_debris_flow',
       'num_events_flash_flood', 'num_events_flood', 'num_events_hail',
       'num_events_heavy_rain', 'num_events_lightning',
       'num_events_rip_current', 'num_events_thunderstorm_wind',
       'num_events_tornado', 'num_events_wildfire',
       'total_crop_damage_debris_flow', 'total_crop_damage_flash_flood',
       'total_crop_damage_flood', 'total_crop_damage_hail',
       'total_crop_damage_heavy_rain', 'total_crop_damage_lightning',
       'total_crop_damage_rip_current', 'total_crop_damage_thunderstorm_wind',
       'total_crop_damage_tornado', 'total_crop_damage_wildfire',
       'total_deaths_debris_flow', 'total_deaths_flash_flood',
       'total_deaths_flood', 'total_deaths_hail', 'total_deaths_heavy_rain',
       'total_deaths_lightning', 'total_deaths_rip_current',
       'total_deaths_thunderstorm_wind', 'total_deaths_tornado',
       'total_deaths_wildfire', 'total_injuries_debris_flow',
       'total_inj

In [10]:
# Aggregate station data by county to merge with ACS data
county_weather = weather_df.groupby(["geo_id", "county_name", "year", "quarter", "county_geometry"]).agg(
    avg_temp=("avg_temp", "mean"),
    max_temp=("max_temp", "max"),
    total_precip=("total_precip", "mean"),
    heat_days_90F=("heat_days_90F", "mean"),
    hail_days=("hail_days", "mean"),
    thunder_days=("thunder_days", "mean"),
    tornado_days=("tornado_days", "mean"),
    reporting_stations=("station_id", "nunique"),
    total_days=("num_days", "mean")
).reset_index()

county_weather.head()

,geo_id,county_name,year,quarter,county_geometry,avg_temp,max_temp,total_precip,heat_days_90F,hail_days,thunder_days,tornado_days,reporting_stations,total_days
0,01003,Baldwin,2006,2,"POLYGON ((-88.02927 30.22271, -88.02399 30.230...",78.274468,96.8,0.000,16.0,0.0,0.0,0.0,1,47.0
1,01003,Baldwin,2006,3,"POLYGON ((-88.02927 30.22271, -88.02399 30.230...",79.402209,95.0,5.665,22.0,0.0,3.0,0.0,2,63.0
2,01003,Baldwin,2006,4,"POLYGON ((-88.02927 30.22271, -88.02399 30.230...",58.596920,89.6,4.635,0.0,0.0,0.0,0.0,2,80.5
3,01003,Baldwin,2007,1,"POLYGON ((-88.02927 30.22271, -88.02399 30.230...",53.967616,82.8,4.005,0.0,0.0,0.0,0.0,2,76.0
4,01003,Baldwin,2007,2,"POLYGON ((-88.02927 30.22271, -88.02399 30.230...",71.111974,95.2,5.695,6.5,0.0,0.0,0.0,2,74.5


In [ ]:
# Merge ACS data with county level quarterly weather data
weather_with_acs = county_weather.merge(
    acs_df,
    on=["geo_id", "year"],
    how="left"
)

# Include severe weather data with the merged df
weather_with_acs = weather_with_acs.merge(storm_wide, on=["geo_id", "year", "quarter"], how="left")

# Fill storm-related NaNs with 0
storm_cols = [col for col in weather_with_acs.columns if "total_" in col or "num_events" in col] # TODO fix total_pop fill
weather_with_acs[storm_cols] = weather_with_acs[storm_cols].fillna(0)

# Flag for downstream modeling since we have more  weather data than acs data currently
weather_with_acs["has_acs"] = weather_with_acs["median_income"].notna().astype(int)

# Training only subset
training_data = weather_with_acs[weather_with_acs["has_acs"] == 1].copy()
training_data.to_csv(DATA_DIR / "county_weather_acs_training.csv", index=False)

# Store complete dataset including nulls
weather_with_acs.to_csv(DATA_DIR / "county_weather_acs_complete.csv", index=False)

print("Merged shape:", weather_with_acs.shape)
print("Training shape:", training_data.shape)
print(weather_with_acs.info())


Merged shape: (142272, 84)
Training shape: (66600, 84)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142272 entries, 0 to 142271
Data columns (total 84 columns):
 #   Column                                   Non-Null Count   Dtype   
---  ------                                   --------------   -----   
 0   geo_id                                   142272 non-null  object  
 1   county_name                              142272 non-null  object  
 2   year                                     142272 non-null  int64   
 3   quarter                                  142272 non-null  int64   
 4   county_geometry                          142272 non-null  geometry
 5   avg_temp                                 142272 non-null  float64 
 6   max_temp                                 142272 non-null  float64 
 7   total_precip                             142272 non-null  float64 
 8   heat_days_90F                            142272 non-null  float64 
 9   hail_days                            